<a href="https://colab.research.google.com/github/DMT130/Costal-Monitoring/blob/main/SAR_Ship_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.dom.minidom
import pandas as pd

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/AI_Datasets/LS-SSDD-v1.0-OPEN.zip

Streaming output truncated to the last 5000 lines.
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_3_9.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_1.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_10.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_11.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_12.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_13.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_14.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_15.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_16.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_17.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_18.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_19.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_2.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_20.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_21.jpg  
  inflating: LS-SSDD-v1.0-OPEN/JPEGImages_sub/07_4_2

In [ ]:
image_path = '/content/LS-SSDD-v1.0-OPEN/JPEGImages_sub'
label_path = '/content/LS-SSDD-v1.0-OPEN/Annotations_sub'

In [ ]:
def xmlvalues(name):
        xminv = []
        yminv = []
        xmaxv = []
        ymaxv = []
        foldervl = []
        filenamevl = []
        widthvl = []
        heightvl = []
        depthvl = []
        clasl =[]
        domtree = xml.dom.minidom.parse(name)
        annotation = domtree.documentElement
        folderv = annotation.getElementsByTagName('folder')[0].childNodes[0].data
        filenamev = annotation.getElementsByTagName('filename')[0].childNodes[0].data.split('.')[0]+'.jpg'
        size = annotation.getElementsByTagName('size')
        for value in size:
            widthv = value.getElementsByTagName('width')[0].childNodes[0].data
            heightv = value.getElementsByTagName('height')[0].childNodes[0].data
            depthv = value.getElementsByTagName('depth')[0].childNodes[0].data

        objects = annotation.getElementsByTagName('object')

        for value in objects:
          clas = value.getElementsByTagName('name')[0].childNodes[0].data
          bbox = value.getElementsByTagName('bndbox')
          for bb in bbox:
              xminv.append(bb.getElementsByTagName('xmin')[0].childNodes[0].data)
              yminv.append(bb.getElementsByTagName('ymin')[0].childNodes[0].data)
              xmaxv.append(bb.getElementsByTagName('xmax')[0].childNodes[0].data)
              ymaxv.append(bb.getElementsByTagName('ymax')[0].childNodes[0].data)
              foldervl.append(folderv)
              filenamevl.append(filenamev)
              widthvl.append(widthv)
              heightvl.append(heightv)
              depthvl.append(depthv)
              clasl.append(clas)

        return pd.DataFrame({'folder':foldervl, 'filename':filenamevl, 'width':widthvl, 'height':heightvl, 'depth':depthvl, 'class':clasl, 'xmin':xminv, 'ymin':yminv, 'xmax':xmaxv, 'ymax':ymaxv})

In [ ]:
def datafrane(listxml):
    df = pd.DataFrame()
    for i in listxml:
      fd = xmlvalues(i)
      fd = pd.concat([fd,df])
      df = fd
    return df

In [ ]:
lablelist = !ls '/content/LS-SSDD-v1.0-OPEN/Annotations_sub'

In [ ]:
lable = []
for i in (lablelist):
  a = i.split( )
  for b in a:
    lable.append(label_path+'/'+b)

In [ ]:
df = datafrane(lable)

In [ ]:
df.head()

,folder,filename,width,height,depth,class,xmin,ymin,xmax,ymax
0,JPEGImages,15_9_9.jpg,800,800,1,ship,584,135,599,149
0,JPEGImages,09_9_9.jpg,800,800,1,ship,720,631,733,668
1,JPEGImages,09_9_9.jpg,800,800,1,ship,257,103,303,129
2,JPEGImages,09_9_9.jpg,800,800,1,ship,305,78,337,105
3,JPEGImages,09_9_9.jpg,800,800,1,ship,327,65,379,94


In [ ]:
!pip install -q --upgrade selectivesearch torch_snippets

     |████████████████████████████████| 56 kB 2.9 MB/s 
     |████████████████████████████████| 235 kB 13.8 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 1.6 MB 44.3 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 


In [ ]:
from torch_snippets import *
from PIL import Image
from torchvision import transforms, models, datasets
from torch_snippets import Report
from torchvision.ops import nms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#Image.MAX_IMAGE_PIXELS = 1933120000

In [ ]:
import glob

In [ ]:
import torch.nn.functional as F

In [ ]:
label2target = {l:t+1 for t,l in enumerate(df['class'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)

In [ ]:
label2target

{'ship': 1, 'background': 0}

In [ ]:
idexis = []
for i in range(len(df['class'])):
  idexis.append(i)

In [ ]:
df['idexis']=idexis

In [ ]:
df.head()

,folder,filename,width,height,depth,class,xmin,ymin,xmax,ymax,idexis
0,JPEGImages,15_9_9.jpg,800,800,1,ship,584,135,599,149,0
0,JPEGImages,09_9_9.jpg,800,800,1,ship,720,631,733,668,1
1,JPEGImages,09_9_9.jpg,800,800,1,ship,257,103,303,129,2
2,JPEGImages,09_9_9.jpg,800,800,1,ship,305,78,337,105,3
3,JPEGImages,09_9_9.jpg,800,800,1,ship,327,65,379,94,4


In [ ]:
#img = Image.open('/content/LS-SSDD-v1.0-OPEN/JPEGImages/01.jpg')

In [ ]:
#plt.imshow(img)

In [ ]:
def tofloat(column):
  l = []
  for i in column:
    i = l.append(int(i))
  return l

In [ ]:
df.xmin = tofloat(df.xmin)
df.ymin = tofloat(df.ymin)
df.xmax = tofloat(df.xmax)
df.ymax = tofloat(df.ymax)

In [ ]:
#df.xmin = df.xmin*224/800
#df.ymin = df.ymin*224/800
#df.xmax = df.xmax*224/800
#df.ymax = df.ymax*224/800

In [ ]:
#df.xmin = tofloat(df.xmin)
#df.ymin = tofloat(df.ymin)
#df.xmax = tofloat(df.xmax)
#df.ymax = tofloat(df.ymax)

In [ ]:
df.head()

,folder,filename,width,height,depth,class,xmin,ymin,xmax,ymax,idexis
0,JPEGImages,15_9_9.jpg,800,800,1,ship,584,135,599,149,0
0,JPEGImages,09_9_9.jpg,800,800,1,ship,720,631,733,668,1
1,JPEGImages,09_9_9.jpg,800,800,1,ship,257,103,303,129,2
2,JPEGImages,09_9_9.jpg,800,800,1,ship,305,78,337,105,3
3,JPEGImages,09_9_9.jpg,800,800,1,ship,327,65,379,94,4


In [ ]:
df['Area']=(df.xmax-df.xmin)*(df.ymax-df.ymin)

In [ ]:
df.head()

,folder,filename,width,height,depth,class,xmin,ymin,xmax,ymax,idexis,Area
0,JPEGImages,15_9_9.jpg,800,800,1,ship,584,135,599,149,0,210
0,JPEGImages,09_9_9.jpg,800,800,1,ship,720,631,733,668,1,481
1,JPEGImages,09_9_9.jpg,800,800,1,ship,257,103,303,129,2,1196
2,JPEGImages,09_9_9.jpg,800,800,1,ship,305,78,337,105,3,864
3,JPEGImages,09_9_9.jpg,800,800,1,ship,327,65,379,94,4,1508


In [ ]:
df=df[df['Area']>0].copy()

In [ ]:
class NoneTransform(object):
    ''' Does nothing to the image. To be used instead of None '''

    def __call__(self, image):
        return image

In [ ]:
def preprocess_image(img):
    img = torch.tensor(img).permute(2,0,1)
    return img.to(device).float()

In [ ]:
class OpenDataset(torch.utils.data.Dataset):
    #w, h = 224, 224
    def __init__(self, df, image_path=image_path):
        self.image_path = image_path
        self.files = glob.glob(self.image_path+'/*')
        self.df = df
        self.image_infos = df.filename.unique()
    def __getitem__(self, ix):
        # load images and masks
        image_id = self.image_infos[ix]
        img_path = find(image_id, self.files)
        img = Image.open(img_path)
        #img.thumbnail((224,224), Image.ANTIALIAS)
        T = transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1))  if img.mode!='RGB'  else NoneTransform()
            ])

        img = T(img)/255.
        img = img.to(device).float()
        #img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.
        data = self.df[self.df['filename'] == image_id]
        labels = data['class'].values.tolist()
        data = data[['xmin','ymin','xmax','ymax']].values
        #data[:,[0,2]] *= self.w
        #data[:,[1,3]] *= self.h
        boxes = data.astype(np.uint32).tolist() # convert to absolute coordinates
        #boxes = data.tolist()
        # torch FRCNN expects ground truths as a dictionary of tensors
        target = {}
        target["boxes"] = torch.Tensor(boxes).float()
        target["labels"] = torch.Tensor([label2target[i] for i in labels]).long()
        #img = preprocess_image(img)
        return img, target
    def collate_fn(self, batch):
        return tuple(zip(*batch))

    def __len__(self):
        return len(self.image_infos)

In [ ]:
from sklearn.model_selection import train_test_split
trn_ids, val_ids = train_test_split(df.filename.unique(), test_size=0.2, random_state=99)
trn_df, val_df = df[df['filename'].isin(trn_ids)], df[df['filename'].isin(val_ids)]
len(trn_df), len(val_df)

train_ds = OpenDataset(trn_df)
test_ds = OpenDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=train_ds.collate_fn, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_fn, drop_last=True)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# Defining training and validation functions for a single batch
def train_batch(inputs, model, optimizer):
    model.train()
    input, targets = inputs
    input = list(image.to(device) for image in input)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    optimizer.zero_grad()
    losses = model(input, targets)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses

@torch.no_grad() # this will disable gradient computation in the function below
def validate_batch(inputs, model):
    model.train() # to obtain the losses, model needs to be in train mode only. # #Note that here we are not defining the model's forward method
#and hence need to work per the way the model class is defined
    input, targets = inputs
    input = list(image.to(device) for image in input)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    optimizer.zero_grad()
    losses = model(input, targets)
    loss = sum(loss for loss in losses.values())
    return loss, losses

In [ ]:
model = get_model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
n_epochs = 10
log = Report(n_epochs)

In [ ]:
for epoch in range(n_epochs):
    _n = len(train_loader)
    for ix, inputs in enumerate(train_loader):
        loss, losses = train_batch(inputs, model, optimizer)
        loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = \
            [losses[k] for k in ['loss_classifier','loss_box_reg','loss_objectness','loss_rpn_box_reg']]
        pos = (epoch + (ix+1)/_n)
        log.record(pos, trn_loss=loss.item(), trn_loc_loss=loc_loss.item(),
                   trn_regr_loss=regr_loss.item(), trn_objectness_loss=loss_objectness.item(),
                   trn_rpn_box_reg_loss=loss_rpn_box_reg.item(), end='\r')

    _n = len(test_loader)
    for ix,inputs in enumerate(test_loader):
        loss, losses = validate_batch(inputs, model)
        loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = \
          [losses[k] for k in ['loss_classifier','loss_box_reg','loss_objectness','loss_rpn_box_reg']]
        pos = (epoch + (ix+1)/_n)
        log.record(pos, val_loss=loss.item(), val_loc_loss=loc_loss.item(),
                  val_regr_loss=regr_loss.item(), val_objectness_loss=loss_objectness.item(),
                  val_rpn_box_reg_loss=loss_rpn_box_reg.item(), end='\r')
    if (epoch+1)%(n_epochs//5)==0: log.report_avgs(epoch+1)

In [ ]:
log.plot_epochs(['trn_loss','val_loss'])

In [ ]:


from torchvision.ops import nms
def decode_output(output):
    'convert tensors to numpy arrays'
    bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
    labels = np.array([target2label[i] for i in output['labels'].cpu().detach().numpy()])
    confs = output['scores'].cpu().detach().numpy()
    ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
    bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

    if len(ixs) == 1:
        bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
    return bbs.tolist(), confs.tolist(), labels.tolist()



In [ ]:
model.eval()
for ix, (images, targets) in enumerate(test_loader):
    if ix==3: break
    images = [im for im in images]
    outputs = model(images)
    for ix, output in enumerate(outputs):
        bbs, confs, labels = decode_output(output)
        info = [f'{l}@{c:.2f}' for l,c in zip(labels, confs)]
        img = images[ix][1]*255
        show(img.cpu(), bbs=bbs, texts=labels, sz=5)

In [ ]:
torch.save(model.state_dict(), 'sarshipv2.pt')

In [ ]:
outpath = '/content/drive/MyDrive/SAR/sarshipv22.pt'

In [ ]:
torch.save(model.state_dict(), outpath)